### Importando Bibliotecas

In [19]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv("credit.csv")
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad


In [4]:
# Dimensões do arquivo
df.shape

(1000, 21)

In [5]:
# Remover valores nulos
df = df.dropna()

# Remover duplicatas
df = df.drop_duplicates()

# Normalizar nomes das colunas
df.columns = df.columns.str.lower().str.replace(' ', '_')




In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   float64
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   float64
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   float64
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   float64
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

In [7]:
df.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [8]:
for col in df:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

In [9]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,1,6.0,1,6,1169.0,4,3,4.0,3,2,...,3,67.0,1,1,2.0,1,1.0,1,1,1
1,0,48.0,3,6,5951.0,2,0,2.0,0,2,...,3,22.0,1,1,1.0,1,1.0,0,1,0
2,3,12.0,1,2,2096.0,2,1,2.0,3,2,...,3,49.0,1,1,1.0,3,2.0,0,1,1
3,1,42.0,3,3,7882.0,2,1,2.0,3,1,...,1,45.0,1,0,1.0,1,2.0,0,1,1
4,1,24.0,2,4,4870.0,2,0,3.0,3,2,...,2,53.0,1,0,2.0,1,2.0,0,1,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   int8   
 1   duration                1000 non-null   float64
 2   credit_history          1000 non-null   int8   
 3   purpose                 1000 non-null   int8   
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   int8   
 6   employment              1000 non-null   int8   
 7   installment_commitment  1000 non-null   float64
 8   personal_status         1000 non-null   int8   
 9   other_parties           1000 non-null   int8   
 10  residence_since         1000 non-null   float64
 11  property_magnitude      1000 non-null   int8   
 12  age                     1000 non-null   float64
 13  other_payment_plans     1000 non-null   int8   
 14  housing                 1000 non-null   i

### Separar em train e test

In [11]:
previsoes = df.iloc[:,0:20].values
classe = df.iloc[:,20:]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(previsoes, classe, test_size=0.3, random_state= 123)

### Treinamento do Algoritmo

In [ ]:
# criar Experimento
mlflow.set_experiment('NB_modulo4_MLOPS')

# run do experimento
with mlflow.start_run():
    naive_bayes = GaussianNB()
    naive_bayes.fit(X_train, y_train)
    previsoes = naive_bayes.predict(X_test)

# metricas 
acuracia = accuracy_score(y_test, pred)
recall =  recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, pred)
log = log_loss(y_test, pred)

# registrar metricas
mlflow.log_metric('acuracia', acuracia)
mlflow.log_metric('recall', recall)
mlflow.log_metric('precision', precision)
ml


c:\Users\adair\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
acuracia = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, pred)
log = log_loss(y_test, pred)

In [18]:
print(f'''Acuracia : {acuracia}
recall : {recall}
Precisão: {precision}
F1: {f1}
AUC: {auc}
LOG: {log}
''')

Acuracia : 0.7166666666666667
recall : 0.77
Precisão: 0.7979274611398963
F1: 0.7837150127226463
AUC: 0.69
LOG: 10.212368460249861

